In [1]:
batch = 'Metaltail'
die = '21'
subdie = '14'
qubit_box = 'QBB46_2_6'
sample_name = 'IMEC-'+batch+'_Die'+die+'_Subdie'+subdie+'_'+qubit_box

# Software imports

In [3]:
import copy 
from functools import partial 
import importlib 
from math import floor
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import scipy
from scipy import signal

import json

import qcodes as qc

from qcodes.utils.dataset.doNd import do1d, do2d, do0d
from qcodes.logger.logger import start_all_logging

import qcodes.dataset.experiment_container as exc
from qcodes.dataset.plotting import plot_by_id, plot_dataset
from qcodes import initialise_database, Measurement, load_by_id, \
    initialise_or_create_database_at, load_or_create_experiment
from qcodes import Parameter
from qcodes.utils.validators import Numbers

from qcodes.instrument_drivers.QDevil.QDevil_QDAC import Mode, ChannelList
from qcodes_contrib_drivers.drivers.QDevil import QDAC2

# Database and STATION setup

In [4]:
sofias_path = os.path.join('F:\\', 'qcodes_local', 'Sofia')

data_path = os.path.join('F:\\', 'qcodes_local', 'Sofia', 'IMEC', 'Metaltail', 
    '10.4. JellyBean-coupled double-DQD with SETs', 'Die21_Subdie14_QBB46_2_6', 'Week 10')

In [5]:
STATION = qc.Station(config_file=os.path.join(data_path, 'Config files', 'config_20220311.yaml'), 
    use_monitor='False') 

c:\Users\T4ACQ\miniconda3\envs\qcodes\lib\site-packages\qcodes\station.py:429: ValidationWarning: Additional properties are not allowed ('auto_reconnect' was unexpected)

Failed validating 'additionalProperties' in schema['properties']['instruments']['patternProperties']['^\\w+$']:
    {'additionalProperties': False,
     'properties': {'add_parameters': {'additionalProperties': False,
                                       'patternProperties': {'^(\\w+)(\\.\\w+)*$': {'additionalProperties': False,
                                                                                    'properties': {'initial_value': {},
                                                                                                   'inter_delay': {'type': 'number'},
                                                                                                   'label': {'type': 'string'},
                                                                                                   'limits': {'add

In [5]:
initialise_or_create_database_at(os.path.join(data_path, 'Experiment databases', 
    sample_name+'_2022_05_15_leakage_tests.db'))

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████| 1/1 [00:00<00:00, 333.41it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████| 1/1 [00:00<00:00, 499.56it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|██████████| 1/1 [00:00<00:00, 1001.27it/s]


In [6]:
initialise_or_create_database_at(os.path.join(data_path, 'Experiment databases', 
    sample_name+'_2022_05_15_leakage_tests.db'))

# Load instruments

### QDac II-1

In [7]:
qdacII1 = QDAC2.QDac2(name='QDacII1', address='ASRL7::INSTR')

Connected to: QDevil QDAC-II (serial:62, firmware:5-0.9.26) in 0.19s


In [34]:
for ch in qdacII1.channels:
    if (np.abs(ch.dc_constant_V() - 0.0) > 1e-5):
        print("ch ",ch._name," voltage: ",ch.dc_constant_V())

In [9]:
for ch in qdacII1.channels:
    ch.output_range('high')
    ch.measurement_range('low')

In [10]:
tstart = time.time()
num_repetitions = 10
for i in range(num_repetitions):
    qdacII1.ch01.read_current_A()
tend = time.time()
print("Elapsed time per current read in QDac2: ", (tend - tstart)/num_repetitions)

Elapsed time per current read in QDac2:  0.02328662872314453


### QDac II-2

In [11]:
qdacII2 = QDAC2.QDac2(name='QDacII2', address='ASRL8::INSTR')

Connected to: QDevil QDAC-II (serial:33, firmware:6-0.13.0) in 0.14s


In [35]:
for ch in qdacII2.channels:
    if (np.abs(ch.dc_constant_V() - 0.0) > 1e-5):
        print("ch ",ch._name," voltage: ",ch.dc_constant_V())

In [13]:
for ch in qdacII2.channels:
    ch.output_range('high')
    ch.measurement_range('low')

In [13]:
tstart = time.time()
num_repetitions = 10
for i in range(num_repetitions):
    qdacII1.ch01.read_current_A()
tend = time.time()
print("Elapsed time per current read in QDac2: ", (tend - tstart)/num_repetitions)

Elapsed time per current read in QDac2:  0.0156510591506958


# Voltage and current channels

In [14]:
filename = 'gate_name_to_qdac_channel_mapping.json'
with open(os.path.join(data_path, 'Config files', filename), 'r') as json_file:
    gate_to_qdac_channel_mapping = json.load(json_file)

print("number of channels connected to qdacII1: ", len(list(gate_to_qdac_channel_mapping['qdacII1'].values())))
print("\nqdacII1 gates: ")
for k, v in gate_to_qdac_channel_mapping['qdacII1'].items():
    print(k,": ",v)

print("\nnumber of channels connected to qdacII2: ", len(list(gate_to_qdac_channel_mapping['qdacII2'].values())))
print("\nqdacII2 gates: ")
for k, v in gate_to_qdac_channel_mapping['qdacII2'].items():
    print(k,": ",v)

number of channels connected to qdacII1:  17

qdacII1 gates: 
top_left_confinement :  2
top_right_confinement :  22
reservoir_gate :  18
reservoir_ohmic :  16
barrier_reservoir_Tplunger :  13
Tplunger :  14
SET1_confinement :  6
SET1_plunger1_gate :  4
SET1_plunger1_ohmic :  1
SET1_right_barrier :  11
SET1_plunger2_gate :  10
SET1_plunger2_ohmic :  7
SET2_confinement :  21
SET2_plunger1_gate :  19
SET2_left_barrier :  20
SET2_plunger2_gate :  23
SET2_plunger2_ohmic :  24

number of channels connected to qdacII2:  18

qdacII2 gates: 
bottom_confinement :  6
source_gate :  20
source_ohmic :  22
barrier_source_plunger1 :  21
plunger1 :  19
barrier_plunger1_plunger2 :  18
plunger2 :  17
barrier_plunger2_Tplunger :  15
barrier_Tplunger_plunger3 :  13
plunger3 :  12
barrier_plunger3_plunger4 :  11
plunger4 :  10
barrier_plunger4_drain :  4
drain_gate :  3
drain_ohmic :  2
SET1_left_barrier :  24
SET2_plunger1_ohmic :  7
SET2_right_barrier :  1


In [17]:
safety_limits = {'V_gate': np.array([-1.0, 3.0]),
                 'V_confinement': np.array([-1.1, 0.1]), 
                 'V_ohmic': np.array([-0.025, 0.025])}

In [16]:
def get_qdacII_current(qdac, channel:int):
    return getattr(qdac, 'ch'+'{:02d}'.format(channel)).read_current_A()[0]

In [18]:
for qdacII1_ch in qdacII1.channels:
    qdacII1_ch_index = int(qdacII1_ch._name.split("ch")[1])
    if (qdacII1_ch_index in gate_to_qdac_channel_mapping['qdacII1'].values()):
        gate_name = list(gate_to_qdac_channel_mapping['qdacII1'].keys())[list(gate_to_qdac_channel_mapping['qdacII1'].values()).index(qdacII1_ch_index)]
        vars()["I_"+gate_name] = qc.Parameter(instrument=qdacII1, name="I_"+gate_name, 
                                              label="I_"+gate_name+"_ch_"+str(qdacII1_ch_index), 
                                              unit="A", 
                                              get_cmd=partial(get_qdacII_current, **{'qdac': qdacII1, 'channel': qdacII1_ch_index})) 
        if ("ohmic" in gate_name):
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII1, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII1_ch_index), 
                                                  unit="V", 
                                                  get_cmd=qdacII1_ch.dc_constant_V, 
                                                  set_cmd=qdacII1_ch.dc_constant_V, vals=Numbers(safety_limits['V_ohmic'].min(), safety_limits['V_ohmic'].max())) 
        elif ("confinement" in gate_name):
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII1, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII1_ch_index), 
                                                  unit="V", 
                                                  get_cmd=qdacII1_ch.dc_constant_V, 
                                                  set_cmd=qdacII1_ch.dc_constant_V, 
                                                  vals=Numbers(safety_limits['V_confinement'].min(), safety_limits['V_confinement'].max())) 
        else:
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII1, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII1_ch_index), 
                                                  unit="V", get_cmd=qdacII1_ch.dc_constant_V, 
                                                  set_cmd=qdacII1_ch.dc_constant_V, 
                                                  vals=Numbers(safety_limits['V_gate'].min(), safety_limits['V_gate'].max())) 

In [20]:
for qdacII2_ch in qdacII2.channels:
    qdacII2_ch_index = int(qdacII2_ch._name.split("ch")[1])
    if (qdacII2_ch_index in gate_to_qdac_channel_mapping['qdacII2'].values()):
        gate_name = list(gate_to_qdac_channel_mapping['qdacII2'].keys())[list(gate_to_qdac_channel_mapping['qdacII2'].values()).index(qdacII2_ch_index)]
        vars()["I_"+gate_name] = qc.Parameter(instrument=qdacII2, name="I_"+gate_name, 
                                              label="I_"+gate_name+"_ch_"+str(qdacII2_ch_index), 
                                              unit="A", 
                                              get_cmd=partial(get_qdacII_current, **{'qdac': qdacII2, 'channel': qdacII2_ch_index})) 
        if ("ohmic" in gate_name):
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII2, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII2_ch_index), 
                                                  unit="V", 
                                                  get_cmd=qdacII2_ch.dc_constant_V, 
                                                  set_cmd=qdacII2_ch.dc_constant_V, vals=Numbers(safety_limits['V_ohmic'].min(), safety_limits['V_ohmic'].max())) 
        elif ("confinement" in gate_name):
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII2, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII2_ch_index), 
                                                  unit="V", 
                                                  get_cmd=qdacII2_ch.dc_constant_V, 
                                                  set_cmd=qdacII2_ch.dc_constant_V, 
                                                  vals=Numbers(safety_limits['V_confinement'].min(), safety_limits['V_confinement'].max())) 
        else:
            vars()["V_"+gate_name] = qc.Parameter(instrument=qdacII2, name="V_"+gate_name, 
                                                  label="V_"+gate_name+"_ch_"+str(qdacII2_ch_index), 
                                                  unit="V", get_cmd=qdacII2_ch.dc_constant_V, 
                                                  set_cmd=qdacII2_ch.dc_constant_V, 
                                                  vals=Numbers(safety_limits['V_gate'].min(), safety_limits['V_gate'].max())) 

In [21]:
current_channels = []
voltage_channels = []
for gate_name, qdacII1_channel in gate_to_qdac_channel_mapping['qdacII1'].items():
    for qdacII1_ch in qdacII1.channels:
        qdacII1_ch_index = int(qdacII1_ch._name.split("ch")[1])
        if (qdacII1_ch_index == qdacII1_channel):
            current_channels.append(vars()["I_"+gate_name])
            voltage_channels.append(vars()["V_"+gate_name])

for gate_name, qdacII2_channel in gate_to_qdac_channel_mapping['qdacII2'].items():
    for qdacII2_ch in qdacII2.channels:
        qdacII2_ch_index = int(qdacII2_ch._name.split("ch")[1])
        if (qdacII2_ch_index == qdacII2_channel):
            current_channels.append(vars()["I_"+gate_name])
            voltage_channels.append(vars()["V_"+gate_name])

In [32]:
for vch in voltage_channels:
    print('{0:>35}'.format(vch.label), '\t', vch._instrument.name)

        V_top_left_confinement_ch_2 	 QDacII1
      V_top_right_confinement_ch_22 	 QDacII1
             V_reservoir_gate_ch_18 	 QDacII1
            V_reservoir_ohmic_ch_16 	 QDacII1
 V_barrier_reservoir_Tplunger_ch_13 	 QDacII1
                   V_Tplunger_ch_14 	 QDacII1
            V_SET1_confinement_ch_6 	 QDacII1
          V_SET1_plunger1_gate_ch_4 	 QDacII1
         V_SET1_plunger1_ohmic_ch_1 	 QDacII1
         V_SET1_right_barrier_ch_11 	 QDacII1
         V_SET1_plunger2_gate_ch_10 	 QDacII1
         V_SET1_plunger2_ohmic_ch_7 	 QDacII1
           V_SET2_confinement_ch_21 	 QDacII1
         V_SET2_plunger1_gate_ch_19 	 QDacII1
          V_SET2_left_barrier_ch_20 	 QDacII1
         V_SET2_plunger2_gate_ch_23 	 QDacII1
        V_SET2_plunger2_ohmic_ch_24 	 QDacII1
          V_bottom_confinement_ch_6 	 QDacII2
                V_source_gate_ch_20 	 QDacII2
               V_source_ohmic_ch_22 	 QDacII2
    V_barrier_source_plunger1_ch_21 	 QDacII2
                   V_plunger1_ch_1

# Leakage matrix

**Notes**
- All 35 contacts are connected from qdac channels to bbox channels
- We use dc-1.9 MHz low-pass-filters on all ohmic and ohmic-connected-gate channels 

**List of channels with dc-1.9 MHz low-pass filter**
- `source_gate`
- `source_ohmic`
- `drain_gate`
- `drain_ohmic`
- `SET1_plunger1_gate`
- `SET1_plunger1_ohmic`
- `SET1_plunger2_gate`
- `SET1_plunger2_ohmic`
- `SET2_plunger1_gate`
- `SET2_plunger1_ohmic`
- `SET2_plunger2_gate`
- `SET2_plunger2_ohmic`

In [33]:
exp = exc.load_or_create_experiment("leakage_matrix", 
    sample_name=sample_name)

In [36]:
for Vch in voltage_channels:
    print("Sweeping Vch: ", Vch.name)
    do1d(Vch, -0.005, 0.005, 51, 0.001, *current_channels) 
    Vch(0.0)

Sweeping Vch:  V_top_left_confinement
Starting experimental run with id: 1. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_top_right_confinement
Starting experimental run with id: 2. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_reservoir_gate
Starting experimental run with id: 3. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_reservoir_ohmic
Starting experimental run with id: 4. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_barrier_reservoir_Tplunger
Starting experimental run with id: 5. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_Tplunger
Starting experimental run with id: 6. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_SET1_confinement
Starting experimental run with id: 7. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_SET1_plunger1_gate
Starting experimental run with id: 8. Using 'qcodes.utils.dataset.doNd.do1d'
Sweeping Vch:  V_SET1_plunger1_ohmic
Starting experimental run with id: 9. Using 'qcodes.utils.dataset

# Switch off device (of not off)